In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from NeuralNetwork import NeuralNetwork, sigmoid
from Dense import Dense, Reshape
from Conv import Conv
from Tensor import Tensor
import matplotlib.pyplot as plt
import pyopencl as cl
import pickle
from IPython.display import clear_output
mnist = pickle.load(open('mnist.pkl','rb'))

x_train, y_train = Tensor(mnist['training_images'].reshape(-1,100,28,28)/255), Tensor(np.array([[0]*i+[1]+[0]*(9-i) for i in mnist['training_labels']]))

model = NeuralNetwork([Conv((14,14),3,0,(1,1),sigmoid,inputShape=(28,28,1)),Conv((10,10),9,0,(1,1),sigmoid), Reshape(-1), Dense(10, sigmoid)])
def plotfunc(loss):
    if len(loss) % 50 == 0:
        plt.plot(loss[50:])
        plt.pause(0.1)
        clear_output(wait=True)

model.train(x_train, y_train, epochs=50, lrate=0.01, plotfunc=plotfunc)

KeyboardInterrupt: 

In [1]:
import numpy as np
from NeuralNetwork import NeuralNetwork, sigmoid
from Dense import Dense, Reshape
from Conv import Conv
from Tensor import Tensor
import matplotlib.pyplot as plt
import pyopencl as cl

model = NeuralNetwork([Conv((3,3),10,0,(1,1),sigmoid, inputShape=(6,6,1)),Conv((3,3),3,0,(1,1),sigmoid),Reshape(-1),Dense(10,sigmoid)])
model.allocateMemory(1)
x = Tensor(np.random.randn(*(1,*model.inputShape)))
y = Tensor(np.random.randn(1,*model.outputShape))
model.gradientDescent(x,y,0)
dw,db = model.testGrads(x,y)

l = 0
plt.scatter(dw[l],model.layers[l].dw.get().flatten())
plt.scatter(db[l],model.layers[l].db.get().flatten())

KeyboardInterrupt: 

In [5]:
l = 3
plt.scatter(dw[l],model.layers[l].dw.get().flatten())
plt.scatter(db[l],model.layers[l].db.get().flatten())

NameError: name 'plt' is not defined

In [2]:
from tests import *
from testsConv import *

batchSize = 10
conv = Conv((3,3),10,0,(1,1),sigmoid, inputShape=(6,6,1))
conv.allocateMemory(batchSize)
ym1 = Tensor(np.random.randn(batchSize,*conv.inputShape))
sigmaOut = Tensor(np.random.randn(batchSize,*conv.outputShape))
Y =  Tensor(np.random.randn(batchSize,*conv.outputShape))
e = Tensor(np.zeros((batchSize,*conv.outputShape)))


test(conv.GPUForwardPropagate,forwardPropagate,(ym1,))
test(conv.activate,sigmoidTest,tuple())
test(conv.computeError,computeError,(Y,e))
test(conv.computedb,computedb,(e,))
test(conv.computeGradients,computeGradients,(ym1,e))
test(conv.computeLocalGradient,computeLocalGradient,(sigmaOut,))
test(conv.learningRule,learningRule,(np.float32(0.1),))


Test Passed forwardPropagate
Test Passed sigmoidTest
Test Passed computeError
Test Passed computedb
Test Passed computeGradients
Test Failed computeLocalGradient
Param Index 11 with name "sigmaIn" was off by 32.507816314697266

Test Failed learningRule
Param Index 7 with name "b" was off by 7.607283592224121



In [ ]:
test(conv.activate,sigmoidTest,tuple())

In [2]:
from tests import parse
kernelFuncion = """
kernel void learningRule(const float lrate,
                        const uint filters,
                        const uint batchSize,
                        const uint wsize,
                        global float *dw,
                        global float *db,
                        global float *w,
                        global float *b){

    uint ind = get_global_id(0);

    float sum = 0;
    for(uint batch=0; batch<batchSize; batch++){
        sum += dw[batch*filters*wsize + ind];
    }
    w[ind] += -sum*lrate /batchSize;

    if(ind % wsize){
        sum = 0;
        int j = ind/wsize;
        for(uint batch=0; batch<batchSize; batch++){
            sum += db[batch*filters + j];
        }
        b[j] += -sum*lrate/batchSize;
    }
}
"""
print(parse(kernelFuncion))


def learningRule(  lrate,
                          filters,
                          batchSize,
                          wsize,
                         *dw,
                         *db,
                         *w,
                         *b):

     ind = globalIndex(0)

     sum = 0
    for( batch=0 batch<batchSize batch++):
        sum += dw[batch*filters*wsize + ind]
    
    w[ind] += -sum*lrate /batchSize

    if(ind % wsize):
        sum = 0
        int j = ind/wsize
        for( batch=0 batch<batchSize batch++):
            sum += db[batch*filters + j]
        
        b[j] += -sum*lrate/batchSize
    




In [3]:
kernelFuncion.replace('kernel void','def')

'\ndef sigmoid(global float *v,\n                    global float *y, global float *dphi){\n \n    uint ind = get_global_id(0);\n    if (v[ind] > 80){\n        v[ind] = 80;\n    }\n    if (v[ind] < -80){\n        v[ind] = -80;\n    }\n    y[ind] = 1/( 1 + exp(-v[ind]) );\n    dphi[ind] = y[ind]*(1-y[ind]);\n}\n'